In [23]:
from mylib import lib
import json
import pandas as pd
pd.get_option("display.max_columns")
pd.get_option("display.max_colwidth")
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_colwidth', 100)

df_techskill = pd.read_csv('../data/technology_skill.csv', header=0)
print(df_techskill.shape)
df_techskill[0:1]

(8743, 2)


,skill_id,skill_name
0,1,!Trak-it Solutions !Trak-it HR


In [24]:
df_jobs = pd.read_csv('../data/Jobs_NYC_Postings.csv', header=0)
df_jobs.rename(columns={"Job ID":'job_id'}, inplace=True)
df_jobs.rename(columns={"Business Title":'job_title'}, inplace=True)
df_jobs.rename(columns={"Job Description":'job_description'}, inplace=True)
df_jobs.rename(columns={"Minimum Qual Requirements":'min_requirements'}, inplace=True)
df_jobs.rename(columns={"Preferred Skills":'preferred_skills'}, inplace=True)
print(df_jobs.shape)
df_jobs[0:1]

(6186, 30)


,job_id,Agency,Posting Type,# Of Positions,job_title,Civil Service Title,Title Classification,Title Code No,Level,Job Category,Full-Time/Part-Time indicator,Career Level,Salary Range From,Salary Range To,Salary Frequency,Work Location,Division/Work Unit,job_description,min_requirements,preferred_skills,Additional Information,To Apply,Hours/Shift,Work Location 1,Recruitment Contact,Residency Requirement,Posting Date,Post Until,Posting Updated,Process Date
0,606346,DEPARTMENT FOR THE AGING,External,1,Business Operations Analyst,ASSOCIATE STAFF ANALYST,Competitive-1,12627,00,"Administration & Human Resources Technology, Data & Innovation Policy, Research & Analysis",F,Experienced (non-manager),70611.0,81203.0,Annual,"2 Lafayette St., N.Y.",Human Resources,The Office of Human Resources (OHR) seeks a highly motivated individual to serve as a Business O...,"1. A masterâs degree from an accredited college or university, accredited by regional, nationa...",â¢ Project management experience and expertise is a plus. â¢ Self-starter; works independently...,NaN,Please be sure to submit a resume & cover letter when applying. All current City Employees may a...,NaN,NaN,NaN,"New York City residency is generally required within 90 days of appointment. However, City Emplo...",09/28/2023,27-DEC-2023,09/29/2023,11/20/2023


In [25]:
import os
import pickle
from openai import AzureOpenAI
    
client = AzureOpenAI(
    api_key=os.getenv("Azure_OPENAI_API_KEY"),  
    api_version="2023-12-01-preview",
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
)

9,000 skill words embeddings costed approximately 15 min with text-embedding-ada-002

In [26]:
technology_skill_embeddings = lib.compute_doc_embeddings(client, df_techskill)
with open(f'../data/technology_skill_embeddings.pkl', 'wb') as f:
	pickle.dump(technology_skill_embeddings, f)
technology_skill_embeddings

In [27]:
with open(f'../data/technology_skill_embeddings.pkl', 'rb') as f:
	technology_skill_embeddings = pickle.load(f)
	
# An example embedding:
example_entry = list(technology_skill_embeddings.items())[0]
print(f"{example_entry[0]} : {example_entry[1][:5]}... ({len(example_entry[1])} entries)")

(1, '!Trak-it Solutions !Trak-it HR') : [-0.021032441407442093, -0.013073653914034367, -0.011152084916830063, -0.030493546277284622, 0.009020891040563583]... (1536 entries)


In [40]:
most_relevant_document_sections = lib.order_document_sections_by_query_similarity(client, df_jobs.loc[0, "job_description"], technology_skill_embeddings)
most_relevant_document_sections[0:20]

[(0.8338901535372804, (5625, 'Oracle HRIS')),
 (0.8243166640202755, (3603, 'Human resources management system HRMS')),
 (0.8242578220101647, (5682, 'OrangeHRM')),
 (0.8218084322189023, (3602, 'Human resource management software HRMS')),
 (0.8189976573140134, (3601, 'Human resource information system (HRIS)')),
 (0.8178595296876666, (5651, 'Oracle PeopleSoft Enterprise Human Resources')),
 (0.817397942364923,
  (5613, 'Oracle E-Business Suite Human Resources Management System')),
 (0.8165512041730082, (1873, 'Consultants in Data Processing HRnet')),
 (0.8149008981012226, (4341, 'Lawson Human Resource Management')),
 (0.8137670379005908, (5654, 'Oracle PeopleSoft Human Capital Management')),
 (0.8109003802362466, (6710, 'SAP ERP Human Capital Management')),
 (0.8092388262000062, (3604, 'Human resources software')),
 (0.8086936037861675,
  (3502, 'HarrisData Human Resources Information System HRIS')),
 (0.8072417081962423,
  (3606, 'Humanic Design Human Resources Management System')),
 (0

In [29]:
print(type(most_relevant_document_sections))

<class 'list'>


6,000 job description embedding-search costed approximately xx min with text-embedding-ada-002

In [46]:
df_output = pd.DataFrame(columns=['similarity_score','technology_skill','job_id_title'])

for index, row in df_jobs[0:].iterrows():
    job_id = row['job_id']
    job_title = row['job_title']
    job_description = row['job_description']
    # print(job_description)
    # min_requirements = row['min_requirements']
    # preferred_skills = row['preferred_skills']

    most_relevant_document_sections = lib.order_document_sections_by_query_similarity(client, job_description, technology_skill_embeddings)

    df_tmp = pd.DataFrame(most_relevant_document_sections, columns=['similarity_score','technology_skill'])
    df_tmp["job_id_title"] = "(" + str(job_id) + ", " + job_title + ")"

    df_output = pd.concat([df_output, df_tmp[0:30]], axis=0, ignore_index=True)

df_output.to_csv('../data/technology_skill_30_by_job.csv', index=False)

In [43]:
df_output[0:3]

,similarity_score,technology_skill,job_id_title
0,0.833890,"(5625, Oracle HRIS)","(606346, Business Operations Analyst)"
1,0.824317,"(3603, Human resources management system HRMS)","(606346, Business Operations Analyst)"
2,0.824258,"(5682, OrangeHRM)","(606346, Business Operations Analyst)"


In [44]:
df_pivot = df_output.pivot_table(index='job_id_title', columns='technology_skill', values='similarity_score', aggfunc='mean', fill_value=0)
df_pivot.to_csv('../data/technology_skill_30_by_job_pivot.csv', index=True)
df_pivot[0:3]

technology_skill,"(81, ADP Employease)","(82, ADP Enterprise HR)","(96, AEC Design Group CADPIPE)","(148, ASL HR Director)","(180, Abraxas Energy Consulting Metrix)","(390, Agiliance Compliance Manager)","(510, Analyze)","(890, Automation Counselors municiPAL)","(1037, Bentley SewerCAD)","(1043, Bentley Transportation Data Manager)","(1044, Bentley WaterCAD)","(1137, Bluewater Management Systems BMCASE)","(1160, Bostonpost Technology Property Manager)","(1210, Budget monitoring systems)","(1225, Business Management Systems Municipal Geographic Management System MGMS)","(1873, Consultants in Data Processing HRnet)","(2059, DHI Water and Environment MIKE SHE)","(2282, Digital Crew Teamwork Project Manager)","(2448, EPA Storm Water Management Model SWMM)","(2449, EPCON International CHEMPRO Engineering Suite)","(2615, Elite Software Energy Audit)","(2651, Emerging Technologies Office Center)","(2676, Enercom Energy Depot for Business)","(2677, Enercom Energy Depot for Homes)","(2678, Energy Efficient Rehab Advisor)",...,"(5625, Oracle HRIS)","(5651, Oracle PeopleSoft Enterprise Human Resources)","(5654, Oracle PeopleSoft Human Capital Management)","(5669, Oracle Self-Service Human Resources)","(5682, OrangeHRM)","(5935, Personnel management software)","(6320, Quantum Software Solutions Quantum Project Manager)","(6507, Regional Energy Deployment System ReEDS)","(6694, SAP Americas mySAP ERP Human Capital Management HCM)","(6707, SAP EHS Management)","(6710, SAP ERP Human Capital Management)","(6711, SAP ERP Operations)","(6743, SEDCAD)","(6819, Sage HRMS)","(7725, Tesseract Human Resources Manager)","(7789, Thermal Analysis Systems The Energy Analyst)","(8085, UniFocus Watson Human Resources Manager)","(8341, Wastewater expert control systems)","(8344, Water Erosion Prediction Project WEPP)","(8345, Water Soil and Hydro-Environmental Decision Support System WATERSHEDSS)","(8347, Water resources databases)","(8350, Waterloo Hydrogeologic AquaChem)","(8352, Waterloo Hydrogeologic FlowPath II)","(8513, Workscape HR Service Center)","(8673, hSenid Business Solutions HRM Enterprise)"
job_id_title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"(571361, Executive Program Manager)",0.775779,0.000000,0.765174,0.000000,0.000000,0.000000,0.000000,0.000000,0.764818,0.765189,0.764961,0.777369,0.000000,0.000000,0.000000,0.000000,0.790763,0.000000,0.785375,0.767935,0.000000,0.000000,0.789289,0.776908,0.000000,...,0.00000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.788195,0.000000,0.770728,0.0000,0.000000,0.767525,0.000000,0.000000,0.000000,0.000000,0.795243,0.779379,0.795929,0.774075,0.78057,0.768961,0.000000,0.000000
"(600579, Analyst Energy Efficiency and Modernization)",0.000000,0.000000,0.000000,0.000000,0.778627,0.781762,0.774062,0.789195,0.000000,0.000000,0.000000,0.000000,0.778839,0.788048,0.778193,0.000000,0.000000,0.776233,0.000000,0.000000,0.781488,0.794241,0.000000,0.000000,0.779098,...,0.00000,0.00000,0.000000,0.000000,0.000000,0.00000,0.774261,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.794068,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
"(606346, Business Operations Analyst)",0.000000,0.796798,0.000000,0.794795,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.816551,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.83389,0.81786,0.813767,0.805127,0.824258,0.78912,0.000000,0.000000,0.789969,0.000000,0.8109,0.804039,0.000000,0.794291,0.791298,0.000000,0.803735,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.786715,0.793259
